In [1]:
%pip install git+https://github.com/keras-team/keras-cv

  Cloning https://github.com/keras-team/keras-cv to /tmp/pip-req-build-yt6ncbx0
  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-cv /tmp/pip-req-build-yt6ncbx0
  Resolved https://github.com/keras-team/keras-cv to commit 94b0a551d03ee9de3d81663e2c1a680cb113f7f1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 10.7 MB/s eta 0:00:00
  Created wheel for keras-cv: filename=keras_cv-0.10.0-py3-none-any.whl size=975496 sha256=983aa2f441cbfccb061a7cd22ff221e3499baff05a994b7d9dc1a2e941b89de8
  Stored in directory: /tmp/pip-ephem-wheel-cache-jx47nvlw/wheels/22/4a/22/95891344ab2777b97e94694fd4a187de0b50b831e9027b0f56
Successfully built keras-cv
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attemp

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_cv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

2025-04-16 14:32:17.326736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744813937.582738      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744813937.654837      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load the CSV file
data_path = '/kaggle/input/ham10000-preprocessed-dataset/balanced and standardized images meta for custom.csv'  # Change this to your CSV file path
image_folder = '/kaggle/input/ham10000-preprocessed-dataset/balanced_and_standardized_images/balanced_and_standardized_images'  # Change this to your image folder path
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)

# Extract labels and encode them
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["dx"])  # Encode the 'dx' column

In [4]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.4, stratify=df['label'], random_state=42)

In [5]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 22529 validated image filenames belonging to 7 classes.
Found 5632 validated image filenames belonging to 7 classes.
Found 18774 validated image filenames belonging to 7 classes.


In [6]:
from tensorflow.keras import layers, models, backend as K

def shuffle_unit(inputs, out_channels, bottleneck_ratio=1, stride=1):
    in_channels = inputs.shape[-1]

    if stride == 1:
        x1 = layers.Lambda(lambda z: z[:, :, :, :in_channels // 2])(inputs)
        x2 = layers.Lambda(lambda z: z[:, :, :, in_channels // 2:])(inputs)
        shortcut = x1
        x = x2
    else:
        shortcut = inputs
        x = inputs

    x = layers.Conv2D(out_channels // 2, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.DepthwiseConv2D(3, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(out_channels // 2, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    if stride == 1:
        out = layers.Concatenate()([shortcut, x])
    else:
        shortcut = layers.AveragePooling2D(pool_size=3, strides=2, padding='same')(shortcut)
        out = layers.Concatenate()([shortcut, x])

    return out

def ShuffleNetV2(input_shape=(96, 96, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(24, 3, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Shuffle units
    for _ in range(3):
        x = shuffle_unit(x, 48)
    for _ in range(7):
        x = shuffle_unit(x, 96, stride=2)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, x)


model = ShuffleNetV2(input_shape=(96, 96, 3), num_classes=len(label_encoder.classes_))
model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])


2025-04-16 14:34:26.461508: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [7]:
# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


705/705 ━━━━━━━━━━━━━━━━━━━━ 366s 488ms/step - accuracy: 0.2280 - loss: 1.8991 - val_accuracy: 0.3001 - val_loss: 1.7078
Epoch 2/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 240s 339ms/step - accuracy: 0.3564 - loss: 1.6122 - val_accuracy: 0.3862 - val_loss: 1.5513
Epoch 3/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 249s 351ms/step - accuracy: 0.3981 - loss: 1.5163 - val_accuracy: 0.4252 - val_loss: 1.4362
Epoch 4/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 248s 349ms/step - accuracy: 0.4326 - loss: 1.4304 - val_accuracy: 0.4599 - val_loss: 1.3646
Epoch 5/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 247s 347ms/step - accuracy: 0.4577 - loss: 1.3645 - val_accuracy: 0.4606 - val_loss: 1.3586
Epoch 6/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 243s 343ms/step - accuracy: 0.4774 - loss: 1.3203 - val_accuracy: 0.4959 - val_loss: 1.2913
Epoch 7/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 249s 350ms/step - accuracy: 0.5060 - loss: 1.2658 - val_accuracy: 0.4986 - val_loss: 1.2543
Epoch 8/20
705/705 ━━━━━━━━━━━━━━━━━━━━ 254s 357ms/step - accuracy: 0.5140 - loss: 1.23

In [8]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

In [9]:
plot_model_training_curve(history)

In [10]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



587/587 ━━━━━━━━━━━━━━━━━━━━ 258s 440ms/step - accuracy: 0.5779 - loss: 1.2570

Test Accuracy: 57.98%


In [11]:
model.save('/kaggle/working/shufflenet_model.keras') 
model.save_weights('/kaggle/working/shufflenet_model_weights.weights.h5') 